In [77]:
import pandas as pd
import numpy as np
import snowflake.connector
import os
import glob
import copy
import warnings
import linecache
import requests
from datetime import date
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import snowflake.connector

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = DeprecationWarning)
    warnings.simplefilter("ignore", category = UserWarning)

In [2]:
def get_oauth_token(svc_username=None, svc_password=None):
    """
    Retrives the authentication token for SBD Snowflake.
    This function automatically requests for username and password from the user through interactive prompts.
    If using service account credentials, they can be passed as svc_username and svc_password.
    Keywords:
    svc_username -- Service account username for which the token should be provided.
    svc_password -- Password corresponding to the service account user.
    Example usage:
    # Authentication with your personal user id.
    from sbd_common_utils.snowflake_utils import get_oauth_token
    access_token = get_oauth_token()
    # Authentication with service account.
    from sbd_common_utils.snowflake_utils import get_oauth_token
    from sbd_common_utils.common_utils import get_service_account_creds
    username, password = get_service_account_creds("/datascience/sandbox/someapp/service-account")
    access_token = get_oauth_token(username, password)
    """

    if svc_username and svc_password:
        username = svc_username
        password = svc_password
    else:
        None


    r = requests.post(
        "https://ssoprod.sbdinc.com/as/token.oauth2",
        data={
            "client_id": "Snowflake",
            "grant_type": "password",
            "username": username,
            "password": password,
            "client_secret": 'f9sq630wmLP6UjpSsOk7kTuP6xccCrSOC4YhE1VdTq3GCupqR7gjYcpuhEGRJ9e0',
            "scope": "session:role-any",
        },
    )
    r.raise_for_status()
    access_token = r.json()["access_token"]
    return access_token

In [3]:
'''
Obtain credentials from credentials.txt where 1st line indicates user id, 
2nd line shows password and 3rd line is associated email address.
'''
userid, password, useremail = linecache.getline('credentials.txt', 1).strip(),  linecache.getline('credentials.txt', 2).strip(),  linecache.getline('credentials.txt', 3).strip()

In [4]:
access_token = get_oauth_token(svc_username = userid, svc_password = password)

In [5]:
def check_table_presence(qualified_table_name):
    access_token = get_oauth_token(svc_username = userid, svc_password = password)
    activesnowflakeconnector = snowflake.connector.connect(account = 'sbd_caspian.us-east-1', 
                                                           authenticator = 'oauth', 
                                                           token = access_token,
                                                           warehouse = 'DEV_AIDA_WH',
                                                           database = 'DEV_AIDA',
                                                           role = 'SC_STRATEGY_ANALYTICS_DEV_RW',
                                                           schema = 'SC_STRATEGY_ANALYTICS'
                                                          )
    cur = activesnowflakeconnector.cursor()
    
    # Extract location data 
    try:
        df = pd.DataFrame(cur.execute(f"desc table {qualified_table_name}"))
        return True
    except:
        return False

In [6]:
def drop_table(qualified_table_name):
    access_token = get_oauth_token(svc_username = userid, svc_password = password)
    activesnowflakeconnector = snowflake.connector.connect(account = 'sbd_caspian.us-east-1', 
                                                           authenticator = 'oauth', 
                                                           token = access_token,
                                                           warehouse = 'DEV_AIDA_WH',
                                                           database = 'DEV_AIDA',
                                                           role = 'SC_STRATEGY_ANALYTICS_DEV_RW',
                                                           schema = 'SC_STRATEGY_ANALYTICS'
                                                          )
    cur = activesnowflakeconnector.cursor()
    
    # Extract location data 
    try:
        df = pd.DataFrame(cur.execute(f"drop table {qualified_table_name}"))
        return True
    except:
        return False

In [7]:
def create_table(df, 
                 table_name, 
                 database = 'DEV_AIDA',
                 schema = 'SC_STRATEGY_ANALYTICS'):
    access_token = get_oauth_token(svc_username = userid, svc_password = password)
    engine = create_engine(URL(account = 'sbd_caspian.us-east-1', 
                 authenticator = 'oauth', 
                 token = access_token,
                 warehouse = 'DEV_AIDA_WH',
                 role = 'SC_STRATEGY_ANALYTICS_DEV_RW',
                 database = database,
                 schema = schema
                ))
    connection = engine.connect()

    qualified_table_name = f'{database}.{schema}.{table_name}'
    presence_flag = check_table_presence(qualified_table_name)
    print(f"presence_flag : {presence_flag}")
    if presence_flag == False:    
        df.to_sql(
            table_name, 
            engine, 
            if_exists = "replace", 
            index = False
        )
       
    else:
        #drop_flag = drop_table(qualified_table_name)
        #print(f"drop_flag : {drop_flag}")
        df.to_sql(
            table_name, 
            engine, 
            if_exists = "append", 
            index = False
        )

    connection.close()
    engine.dispose()

In [118]:
df = pd.read_excel("2024.2.19_UnitCostCheck.xlsx", sheet_name='Sheet1')
df.head()

,ProdID,prod_desc,Plnt,super_sbu_gpp,gpp_sbu_desc,gpp_division_desc,CostType,System,Material,Labor,VOH,FOH,TotalOH,Freight,Duty,TotalUnitCost
0,DWE402,4-1/2IN 11 AMP ANGLE GRINDER PADDLE,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1993.9805,71.6705,0.0,0.0,0.0,43.2856,151.9153,2502.7189
1,DWE4120-B3,900W 4-1/2 SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1915.1896,29.9431,0.0,0.0,0.0,56.1868,197.2875,2296.1727
2,DWE402N,4-1/2IN 11 AMP AN GRINDER PADDLE NO-LK,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1990.9941,71.6705,0.0,0.0,0.0,44.7515,157.0533,2506.3364
3,DW840,180MM 1800W MAG 110V,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,3238.8217,47.0016,0.0,0.0,0.0,104.2593,366.4191,3909.6501
4,DWE4212-B3,1200W 5 IN SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,2099.9333,71.6705,0.0,0.0,0.0,44.5912,156.5458,2614.6078


In [119]:
df_analysis = pd.read_csv("cost_analysis_3.csv")
df_analysis.head()

,PROD_KEY,LOC_KEY,SRC_SYS_KEY,MATERIAL_COST,FREIGHT_COST,LABOR_COST,TOTAL_OH_COST,FIXED_OH_COST,DUTY_COST,MIN_MATERIAL_COST,MIN_FREIGHT_COST,MIN_LABOR_COST,MIN_TOTAL_OH_COST,MIN_FIXED_OH_COST,MIN_DUTY_COST
0,DWE4517,73,C11,NaN,NaN,NaN,NaN,NaN,NaN,46.8003,4.5984,2.0061,1.3967,1.1858,3.9313
1,JCS-1046MAS,0084,C11,NaN,NaN,NaN,NaN,NaN,NaN,4175.5929,75.7393,210.1376,2452.5430,1812.1808,2.2923
2,CMEG200,S047,C11,NaN,NaN,NaN,NaN,NaN,NaN,3.3223,0.3216,1.8193,1.5030,1.2761,0.2791
3,J6222CERT,C035,C11,NaN,NaN,NaN,NaN,NaN,NaN,825.3100,41.9258,NaN,NaN,NaN,32.1871
4,J6222CERT,S052,C11,NaN,NaN,NaN,NaN,NaN,NaN,825.3100,41.9258,NaN,NaN,NaN,32.1871


In [120]:
df_analysis.shape, df.shape

((39560, 15), (232, 16))

In [121]:
df_analysis['LOC_KEY'] = df_analysis['LOC_KEY'].apply(lambda x : x.lstrip('0'))

In [122]:
df['Plnt'] = df['Plnt'].astype(str)

In [123]:
prod_plant_list = df[['ProdID', 'Plnt']].values.tolist()

In [124]:
df_analysis_set = pd.DataFrame()
for e in prod_plant_list:
    temp_df_1 = df_analysis[(df_analysis['PROD_KEY'] == e[0]) & (df_analysis['LOC_KEY'] == e[1])]#[['PROD_KEY', 'LOC_KEY']]
    temp_df_2 = df[(df['ProdID'] == e[0]) & (df['Plnt'] == e[1])]#[['PROD_KEY', 'LOC_KEY']]
    temp_df = pd.merge(temp_df_2, temp_df_1, how = 'inner', left_on = ['ProdID', 'Plnt', 'System'], right_on = ['PROD_KEY', 'LOC_KEY', 'SRC_SYS_KEY'])
    df_analysis_set = pd.concat([df_analysis_set, temp_df], ignore_index=True, axis = 0)
df_analysis_set.shape

(232, 31)

In [125]:
df_analysis_set.columns

Index(['ProdID', 'prod_desc', 'Plnt', 'super_sbu_gpp', 'gpp_sbu_desc',
       'gpp_division_desc', 'CostType', 'System', 'Material', 'Labor', 'VOH',
       'FOH', 'TotalOH', 'Freight', 'Duty', 'TotalUnitCost', 'PROD_KEY',
       'LOC_KEY', 'SRC_SYS_KEY', 'MATERIAL_COST', 'FREIGHT_COST', 'LABOR_COST',
       'TOTAL_OH_COST', 'FIXED_OH_COST', 'DUTY_COST', 'MIN_MATERIAL_COST',
       'MIN_FREIGHT_COST', 'MIN_LABOR_COST', 'MIN_TOTAL_OH_COST',
       'MIN_FIXED_OH_COST', 'MIN_DUTY_COST'],
      dtype='object')

In [126]:
df_analysis_set.columns = [col.upper() for col in df_analysis_set.columns]

In [127]:
df_analysis_set.columns

Index(['PRODID', 'PROD_DESC', 'PLNT', 'SUPER_SBU_GPP', 'GPP_SBU_DESC',
       'GPP_DIVISION_DESC', 'COSTTYPE', 'SYSTEM', 'MATERIAL', 'LABOR', 'VOH',
       'FOH', 'TOTALOH', 'FREIGHT', 'DUTY', 'TOTALUNITCOST', 'PROD_KEY',
       'LOC_KEY', 'SRC_SYS_KEY', 'MATERIAL_COST', 'FREIGHT_COST', 'LABOR_COST',
       'TOTAL_OH_COST', 'FIXED_OH_COST', 'DUTY_COST', 'MIN_MATERIAL_COST',
       'MIN_FREIGHT_COST', 'MIN_LABOR_COST', 'MIN_TOTAL_OH_COST',
       'MIN_FIXED_OH_COST', 'MIN_DUTY_COST'],
      dtype='object')

In [128]:
df_analysis_set = df_analysis_set[['PRODID', 'PROD_DESC', 'PLNT', 'SUPER_SBU_GPP', 'GPP_SBU_DESC',
       'GPP_DIVISION_DESC', 'COSTTYPE', 'SYSTEM', 'MATERIAL', 'LABOR', 'VOH',
       'FOH', 'TOTALOH', 'FREIGHT', 'DUTY', 'TOTALUNITCOST', 'MATERIAL_COST', 'FREIGHT_COST', 'LABOR_COST',
       'TOTAL_OH_COST', 'FIXED_OH_COST', 'DUTY_COST', 'MIN_MATERIAL_COST',
       'MIN_FREIGHT_COST', 'MIN_LABOR_COST', 'MIN_TOTAL_OH_COST',
       'MIN_FIXED_OH_COST', 'MIN_DUTY_COST']]


In [129]:
df_analysis_set[['COSTTYPE']].drop_duplicates()

,COSTTYPE
0,Current


In [130]:
df_analysis_set[['SYSTEM']].drop_duplicates()

,SYSTEM
0,E03
5,C11


In [131]:
df_analysis_set['MATERIAL_UNIT_COST_CORRECTED'] = 0
df_analysis_set['LABOR_UNIT_COST_CORRECTED'] = 0
df_analysis_set['VOH_UNIT_COST_CORRECTED'] = 0
df_analysis_set['FOH_UNIT_COST_CORRECTED'] = 0
df_analysis_set['TOTAL_OH_UNIT_COST_CORRECTED'] = 0
df_analysis_set['FREIGHT_UNIT_COST_CORRECTED'] = 0
df_analysis_set['DUTY_UNIT_COST_CORRECTED'] = 0
df_analysis_set['TOTAL_UNIT_COST_CORRECTED'] = 0

In [132]:
df_analysis_set.columns

Index(['PRODID', 'PROD_DESC', 'PLNT', 'SUPER_SBU_GPP', 'GPP_SBU_DESC',
       'GPP_DIVISION_DESC', 'COSTTYPE', 'SYSTEM', 'MATERIAL', 'LABOR', 'VOH',
       'FOH', 'TOTALOH', 'FREIGHT', 'DUTY', 'TOTALUNITCOST', 'MATERIAL_COST',
       'FREIGHT_COST', 'LABOR_COST', 'TOTAL_OH_COST', 'FIXED_OH_COST',
       'DUTY_COST', 'MIN_MATERIAL_COST', 'MIN_FREIGHT_COST', 'MIN_LABOR_COST',
       'MIN_TOTAL_OH_COST', 'MIN_FIXED_OH_COST', 'MIN_DUTY_COST',
       'MATERIAL_UNIT_COST_CORRECTED', 'LABOR_UNIT_COST_CORRECTED',
       'VOH_UNIT_COST_CORRECTED', 'FOH_UNIT_COST_CORRECTED',
       'TOTAL_OH_UNIT_COST_CORRECTED', 'FREIGHT_UNIT_COST_CORRECTED',
       'DUTY_UNIT_COST_CORRECTED', 'TOTAL_UNIT_COST_CORRECTED'],
      dtype='object')

In [133]:
df_analysis_set['MATERIAL_UNIT_COST_CORRECTED'] = df_analysis_set['MIN_MATERIAL_COST']

df_analysis_set['FREIGHT_UNIT_COST_CORRECTED'] = df_analysis_set['MIN_FREIGHT_COST']

df_analysis_set['LABOR_UNIT_COST_CORRECTED'] = df_analysis_set['MIN_LABOR_COST']

df_analysis_set['DUTY_UNIT_COST_CORRECTED'] = df_analysis_set['MIN_DUTY_COST']

df_analysis_set['FOH_UNIT_COST_CORRECTED'] = df_analysis_set['FOH_UNIT_COST_CORRECTED']

df_analysis_set['TOTAL_OH_UNIT_COST_CORRECTED'] = df_analysis_set['TOTAL_OH_UNIT_COST_CORRECTED']

df_analysis_set['TOTAL_UNIT_COST_CORRECTED'] = df_analysis_set[['MATERIAL_UNIT_COST_CORRECTED',
                                                                'FREIGHT_UNIT_COST_CORRECTED',
                                                                'LABOR_UNIT_COST_CORRECTED',
                                                                'DUTY_UNIT_COST_CORRECTED',
                                                                'FOH_UNIT_COST_CORRECTED',
                                                                'TOTAL_OH_UNIT_COST_CORRECTED'
                                                               ]].sum(axis = 1)

In [134]:
df_analysis_set.head()

,PRODID,PROD_DESC,PLNT,SUPER_SBU_GPP,GPP_SBU_DESC,GPP_DIVISION_DESC,COSTTYPE,SYSTEM,MATERIAL,LABOR,...,MIN_FIXED_OH_COST,MIN_DUTY_COST,MATERIAL_UNIT_COST_CORRECTED,LABOR_UNIT_COST_CORRECTED,VOH_UNIT_COST_CORRECTED,FOH_UNIT_COST_CORRECTED,TOTAL_OH_UNIT_COST_CORRECTED,FREIGHT_UNIT_COST_CORRECTED,DUTY_UNIT_COST_CORRECTED,TOTAL_UNIT_COST_CORRECTED
0,DWE402,4-1/2IN 11 AMP ANGLE GRINDER PADDLE,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1993.9805,71.6705,...,241.8670,151.9153,36.0621,71.6705,0,0,0,43.2856,151.9153,302.9335
1,DWE4120-B3,900W 4-1/2 SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1915.1896,29.9431,...,97.5657,197.2875,26.6242,29.9431,0,0,0,56.1868,197.2875,310.0416
2,DWE402N,4-1/2IN 11 AMP AN GRINDER PADDLE NO-LK,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1990.9941,71.6705,...,241.8670,157.0533,50.6219,71.6705,0,0,0,44.7515,157.0533,324.0972
3,DW840,180MM 1800W MAG 110V,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,3238.8217,47.0016,...,153.1484,366.4191,3238.8217,47.0016,0,0,0,104.2593,366.4191,3756.5017
4,DWE4212-B3,1200W 5 IN SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,2099.9333,71.6705,...,241.8670,156.5458,31.3595,71.6705,0,0,0,44.5912,156.5458,304.1670


In [135]:
df_analysis_set.columns

Index(['PRODID', 'PROD_DESC', 'PLNT', 'SUPER_SBU_GPP', 'GPP_SBU_DESC',
       'GPP_DIVISION_DESC', 'COSTTYPE', 'SYSTEM', 'MATERIAL', 'LABOR', 'VOH',
       'FOH', 'TOTALOH', 'FREIGHT', 'DUTY', 'TOTALUNITCOST', 'MATERIAL_COST',
       'FREIGHT_COST', 'LABOR_COST', 'TOTAL_OH_COST', 'FIXED_OH_COST',
       'DUTY_COST', 'MIN_MATERIAL_COST', 'MIN_FREIGHT_COST', 'MIN_LABOR_COST',
       'MIN_TOTAL_OH_COST', 'MIN_FIXED_OH_COST', 'MIN_DUTY_COST',
       'MATERIAL_UNIT_COST_CORRECTED', 'LABOR_UNIT_COST_CORRECTED',
       'VOH_UNIT_COST_CORRECTED', 'FOH_UNIT_COST_CORRECTED',
       'TOTAL_OH_UNIT_COST_CORRECTED', 'FREIGHT_UNIT_COST_CORRECTED',
       'DUTY_UNIT_COST_CORRECTED', 'TOTAL_UNIT_COST_CORRECTED'],
      dtype='object')

In [136]:
df_analysis_set['MATERIAL_COST_CORRECT_FLAG'] = np.where(df_analysis_set['MATERIAL'] > df_analysis_set['MATERIAL_UNIT_COST_CORRECTED'] * .8, 
                                                         'N', 
                                                         'Y'
                                                        )

df_analysis_set['LABOR_COST_CORRECT_FLAG'] = np.where(df_analysis_set['LABOR'] > df_analysis_set['LABOR_UNIT_COST_CORRECTED'] * .8, 
                                                         'N', 
                                                         'Y'
                                                        )

df_analysis_set['VOH_COST_CORRECT_FLAG'] = np.where(df_analysis_set['VOH'] > df_analysis_set['VOH_UNIT_COST_CORRECTED'] * .8, 
                                                         'N', 
                                                         'Y'
                                                        )

df_analysis_set['FOH_COST_CORRECT_FLAG'] = np.where(df_analysis_set['FOH'] > df_analysis_set['FOH_UNIT_COST_CORRECTED'] * .8, 
                                                         'N', 
                                                         'Y'
                                                        )

df_analysis_set['TOTAL_OH_COST_CORRECT_FLAG'] = np.where(df_analysis_set['TOTALOH'] > df_analysis_set['TOTAL_OH_UNIT_COST_CORRECTED'] * .8, 
                                                         'N', 
                                                         'Y'
                                                        )

df_analysis_set['LABOR_COST_CORRECT_FLAG'] = np.where(df_analysis_set['LABOR'] > df_analysis_set['LABOR_UNIT_COST_CORRECTED'] * .8, 
                                                         'N', 
                                                         'Y'
                                                        )

In [137]:
df_analysis_set.head()

,PRODID,PROD_DESC,PLNT,SUPER_SBU_GPP,GPP_SBU_DESC,GPP_DIVISION_DESC,COSTTYPE,SYSTEM,MATERIAL,LABOR,...,FOH_UNIT_COST_CORRECTED,TOTAL_OH_UNIT_COST_CORRECTED,FREIGHT_UNIT_COST_CORRECTED,DUTY_UNIT_COST_CORRECTED,TOTAL_UNIT_COST_CORRECTED,MATERIAL_COST_CORRECT_FLAG,LABOR_COST_CORRECT_FLAG,VOH_COST_CORRECT_FLAG,FOH_COST_CORRECT_FLAG,TOTAL_OH_COST_CORRECT_FLAG
0,DWE402,4-1/2IN 11 AMP ANGLE GRINDER PADDLE,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1993.9805,71.6705,...,0,0,43.2856,151.9153,302.9335,N,N,Y,Y,Y
1,DWE4120-B3,900W 4-1/2 SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1915.1896,29.9431,...,0,0,56.1868,197.2875,310.0416,N,N,Y,Y,Y
2,DWE402N,4-1/2IN 11 AMP AN GRINDER PADDLE NO-LK,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1990.9941,71.6705,...,0,0,44.7515,157.0533,324.0972,N,N,Y,Y,Y
3,DW840,180MM 1800W MAG 110V,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,3238.8217,47.0016,...,0,0,104.2593,366.4191,3756.5017,N,N,Y,Y,Y
4,DWE4212-B3,1200W 5 IN SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,2099.9333,71.6705,...,0,0,44.5912,156.5458,304.1670,N,N,Y,Y,Y


In [138]:
df_analysis_set.iloc[0]

PRODID                                                       DWE402
PROD_DESC                       4-1/2IN 11 AMP ANGLE GRINDER PADDLE
PLNT                                                           7280
SUPER_SBU_GPP                                                   PTG
GPP_SBU_DESC                                       PROF POWER TOOLS
GPP_DIVISION_DESC                    MEP/METALWORKING/DRYWALL (PPT)
COSTTYPE                                                    Current
SYSTEM                                                          E03
MATERIAL                                                  1993.9805
LABOR                                                       71.6705
VOH                                                             0.0
FOH                                                             0.0
TOTALOH                                                         0.0
FREIGHT                                                     43.2856
DUTY                                            

In [139]:
df_analysis_set.columns

Index(['PRODID', 'PROD_DESC', 'PLNT', 'SUPER_SBU_GPP', 'GPP_SBU_DESC',
       'GPP_DIVISION_DESC', 'COSTTYPE', 'SYSTEM', 'MATERIAL', 'LABOR', 'VOH',
       'FOH', 'TOTALOH', 'FREIGHT', 'DUTY', 'TOTALUNITCOST', 'MATERIAL_COST',
       'FREIGHT_COST', 'LABOR_COST', 'TOTAL_OH_COST', 'FIXED_OH_COST',
       'DUTY_COST', 'MIN_MATERIAL_COST', 'MIN_FREIGHT_COST', 'MIN_LABOR_COST',
       'MIN_TOTAL_OH_COST', 'MIN_FIXED_OH_COST', 'MIN_DUTY_COST',
       'MATERIAL_UNIT_COST_CORRECTED', 'LABOR_UNIT_COST_CORRECTED',
       'VOH_UNIT_COST_CORRECTED', 'FOH_UNIT_COST_CORRECTED',
       'TOTAL_OH_UNIT_COST_CORRECTED', 'FREIGHT_UNIT_COST_CORRECTED',
       'DUTY_UNIT_COST_CORRECTED', 'TOTAL_UNIT_COST_CORRECTED',
       'MATERIAL_COST_CORRECT_FLAG', 'LABOR_COST_CORRECT_FLAG',
       'VOH_COST_CORRECT_FLAG', 'FOH_COST_CORRECT_FLAG',
       'TOTAL_OH_COST_CORRECT_FLAG'],
      dtype='object')

In [140]:
df_analysis_set = df_analysis_set[['PRODID', 'PROD_DESC', 'PLNT', 'SUPER_SBU_GPP', 'GPP_SBU_DESC',
       'GPP_DIVISION_DESC', 'COSTTYPE', 'SYSTEM', 'MATERIAL', 'LABOR', 'VOH',
       'FOH', 'TOTALOH', 'FREIGHT', 'DUTY', 'TOTALUNITCOST', 'MATERIAL_COST',
       'FREIGHT_COST', 'LABOR_COST', 'TOTAL_OH_COST', 'FIXED_OH_COST',
       'DUTY_COST','MATERIAL_UNIT_COST_CORRECTED', 'LABOR_UNIT_COST_CORRECTED',
       'VOH_UNIT_COST_CORRECTED', 'FOH_UNIT_COST_CORRECTED',
       'TOTAL_OH_UNIT_COST_CORRECTED', 'FREIGHT_UNIT_COST_CORRECTED',
       'DUTY_UNIT_COST_CORRECTED', 'TOTAL_UNIT_COST_CORRECTED',
       'MATERIAL_COST_CORRECT_FLAG', 'LABOR_COST_CORRECT_FLAG',
       'VOH_COST_CORRECT_FLAG', 'FOH_COST_CORRECT_FLAG',
       'TOTAL_OH_COST_CORRECT_FLAG']]

In [141]:
df_analysis_set.head()

,PRODID,PROD_DESC,PLNT,SUPER_SBU_GPP,GPP_SBU_DESC,GPP_DIVISION_DESC,COSTTYPE,SYSTEM,MATERIAL,LABOR,...,FOH_UNIT_COST_CORRECTED,TOTAL_OH_UNIT_COST_CORRECTED,FREIGHT_UNIT_COST_CORRECTED,DUTY_UNIT_COST_CORRECTED,TOTAL_UNIT_COST_CORRECTED,MATERIAL_COST_CORRECT_FLAG,LABOR_COST_CORRECT_FLAG,VOH_COST_CORRECT_FLAG,FOH_COST_CORRECT_FLAG,TOTAL_OH_COST_CORRECT_FLAG
0,DWE402,4-1/2IN 11 AMP ANGLE GRINDER PADDLE,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1993.9805,71.6705,...,0,0,43.2856,151.9153,302.9335,N,N,Y,Y,Y
1,DWE4120-B3,900W 4-1/2 SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1915.1896,29.9431,...,0,0,56.1868,197.2875,310.0416,N,N,Y,Y,Y
2,DWE402N,4-1/2IN 11 AMP AN GRINDER PADDLE NO-LK,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,1990.9941,71.6705,...,0,0,44.7515,157.0533,324.0972,N,N,Y,Y,Y
3,DW840,180MM 1800W MAG 110V,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,3238.8217,47.0016,...,0,0,104.2593,366.4191,3756.5017,N,N,Y,Y,Y
4,DWE4212-B3,1200W 5 IN SMALL ANGLE GRINDER,7280,PTG,PROF POWER TOOLS,MEP/METALWORKING/DRYWALL (PPT),Current,E03,2099.9333,71.6705,...,0,0,44.5912,156.5458,304.1670,N,N,Y,Y,Y


In [142]:
df_analysis_set.to_csv("cost_analysis_details.csv", index = False)